# KNN with HOG

Needed packages

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from preprocessing.hog import Flatten

Adding our own packages to the system path so that they can be found.

In [39]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Loading Data

We load the hog metadata file in order to find the paths to the training and testing data.

In [42]:
hog_metadata_path = '../../data/hog/metadata.csv'
hog_metadata = pd.read_csv(hog_metadata_path, index_col=0)
hog_metadata.channels = hog_metadata.channels.astype(str)

Train and test data for channels 1,2,3.

In [4]:
channel_1_2_3 = hog_metadata[(hog_metadata.type == 'train') & (hog_metadata.channels == '[1, 2, 3]')]
channel_1_2_3_train = np.load(f'../../{channel_1_2_3.data_path.values[0]}')
channel_1_2_3_train_label = np.load(f'../../{channel_1_2_3.label_path.values[0]}')

channel_1_2_3 = hog_metadata[(hog_metadata.type == 'test') & (hog_metadata.channels == '[1, 2, 3]')]
channel_1_2_3_test = np.load(f'../../{channel_1_2_3.data_path.values[0]}')
channel_1_2_3_test_label = np.load(f'../../{channel_1_2_3.label_path.values[0]}')

Train and test data for channels 6, 7, 8.

In [6]:
channel_6_7_8 = hog_metadata[(hog_metadata.type == 'train') & (hog_metadata.channels == '[6, 7, 8]')]
channel_6_7_8_train = np.load(f'../../{channel_6_7_8.data_path.values[0]}')
channel_6_7_8_train_label = np.load(f'../../{channel_6_7_8.label_path.values[0]}')

channel_6_7_8 = hog_metadata[(hog_metadata.type == 'test') & (hog_metadata.channels == '[6, 7, 8]')]
channel_6_7_8_test = np.load(f'../../{channel_6_7_8.data_path.values[0]}')
channel_6_7_8_test_label = np.load(f'../../{channel_6_7_8.label_path.values[0]}')

Train and test data for channels 11,12,13.

In [9]:
channel_11_12_13 = hog_metadata[(hog_metadata.type == 'train') & (hog_metadata.channels == '[11, 12, 13]')]
channel_11_12_13_train = np.load(f'../../{channel_11_12_13.data_path.values[0]}')
channel_11_12_13_train_label = np.load(f'../../{channel_11_12_13.label_path.values[0]}')

channel_11_12_13 = hog_metadata[(hog_metadata.type == 'test') & (hog_metadata.channels == '[11, 12, 13]')]
channel_11_12_13_test = np.load(f'../../{channel_11_12_13.data_path.values[0]}')
channel_11_12_13_test_label = np.load(f'../../{channel_11_12_13.label_path.values[0]}')

## Creating the KNN Pipeline

In [37]:
pipeline = Pipeline([('flatten', Flatten()), ('knn', KNeighborsClassifier(n_neighbors=7))])

## Scoring Channel 1, 2, 3

In [38]:
pipeline.fit(channel_1_2_3_train, channel_1_2_3_train_label)
pipeline.score(channel_1_2_3_test, channel_1_2_3_test_label)

0.5363636363636364